In [ ]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [12]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'NUGET_XMLDOC_MODE': 'skip',
        'COLORTERM': 'truecolor',
        'CLOUDENV_ENVIRONMENT_ID': 'a681b53b-7dec-453a-9459-cbda762db643',
        'NVM_INC': '/usr/local/share/nvm/versions/node/v20.14.0/include/node',
        'TERM_PROGRAM_VERSION': '1.90.2',
        'GITHUB_USER': 'KOSETTPAI',
        'rvm_prefix': '/usr/local',
        'CODESPACE_NAME': 'studious-carnival-jgqwqwx6x7rf55g5',
        'HOSTNAME': 'codespaces-341ed5',
        'JAVA_ROOT': '/home/codespace/java',
        'JAVA_HOME': '/usr/local/sdkman/candidates/java/current',
        'DOTNET_ROOT': '/usr/share/dotnet',
        'CODESPACES': 'true',
        'PYTHON_ROOT': '/home/codespace/.python',
        'GRADLE_HOME': '/usr/local/sdkman/candidates/gradle/current',
        'NVS_DIR': '/usr/local/nvs',
        'NVS_OS': 'linux',
        'DOTNET_SKIP_FIRST_TIME_EXPERIENCE': '1',
        'MY_RUBY_HOME': '/usr/local/rvm/rubies/ruby-3.2.4',
        'OPENAI_API_KEY': 'sk-Try7nXQMozSyDMQl3

In [13]:
import minsearch

In [14]:
import json

In [15]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [16]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [17]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [18]:
q = 'the course has already started, can I still enroll?'

In [19]:
index.fit(documents)

In [20]:
from openai import OpenAI

In [21]:
client = OpenAI()

In [24]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course' : 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
        
    CONTEXT: {context}
    """
    context = ""
        
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        prompt = prompt_template.format(question=query, context=context).strip()
        return prompt

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages = [{'role' : 'user', 'content' : prompt}]
    )
    return response.choices[0].message.content

In [27]:
query = 'how I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
rag(query)

'To run Kafka in the terminal, navigate to your project directory and execute the following command:\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [29]:
rag('the course is already started. Can I still enroll?')

'Yes, even if the course has already started, you are still eligible to enroll and submit the homework. However, please be mindful of the deadlines for turning in the final projects.'

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [34]:
from elasticsearch import Elasticsearch

In [37]:
es_client = Elasticsearch('http://localhost:9200')

In [38]:
es_client.info()

ObjectApiResponse({'name': '69b212f57fd4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hKZ5I5_iSnKQnJMzl3rYNw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [39]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [40]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [41]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████| 1017/1017 [00:25<00:00, 40.65it/s]


In [44]:
query = 'I just disovered the course. Can I still join it?'

In [45]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

In [47]:
response = es_client.search(index=index_name, body=search_query)

In [53]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [55]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [56]:
rag(query)

'Yes, you can still join the course even after the start date. You are eligible to submit the homeworks, but please be mindful of the deadlines for turning in the final projects, and try not to leave everything for the last minute.'